# part 1. Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto

# 1. import libraries

In [1]:
import pandas as pd #to handle data
import numpy as np # to handle data in vectorized manner
import urllib.request # to handle requests
from bs4 import BeautifulSoup #to parse html documents
import folium #for creating maps
import requests #for handling requests
from sklearn.cluster import KMeans #
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# 2. Scrap data from wikipedia page into a  pandas data frame 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = urllib.request.urlopen(url).read()
soup = BeautifulSoup(page)

In [3]:
# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

In [6]:
for tr in soup.find_all('tr')[2:]:
    tds = tr.find_all('td')
    if len(tds)==1: #To ignore empty rows
        break
    if tds[1].text=='Not assigned': #To ignore values whose borough in "Not assigned"
        continue
    pc=tds[0].text 
    borough=tds[1].text
    print(pc,borough)
    neigh=tds[2].text.rstrip('\n') # rstrip used to ignore extra line
    if neigh=='Not assigned':
        if borough=='Not assigned': # ignoring rows whose neighbor and borough are not assigned
            continue
        else:
            neigh=borough # if neighbor is Not assigned then it is assigned to borough
    # appending values to data frame
    df=df.append({
        'Postcode':pc,
        'Borough':borough,
        'Neighborhood':neigh},ignore_index=True)
df.head()


M1CScarborough(Rouge Hill / Port Union / Highland Creek)

 
M2CNot assigned



M1EScarborough(Guildwood / Morningside / West Hill)

 
M2ENot assigned



M1GScarborough(Woburn)

 
M2GNot assigned



M1HScarborough(Cedarbrae)

 
M2HNorth York(Hillcrest Village)



M1JScarborough(Scarborough Village)

 
M2JNorth York(Fairview / Henry Farm / Oriole)



M1KScarborough(Kennedy Park / Ionview / East Birchmount Park)

 
M2KNorth York(Bayview Village)



M1LScarborough(Golden Mile / Clairlea / Oakridge)

 
M2LNorth York(York Mills / Silver Hills)



M1MScarborough(Cliffside / Cliffcrest / Scarborough Village West)

 
M2MNorth York(Willowdale / Newtonbrook)



M1NScarborough(Birch Cliff / Cliffside West)

 
M2NNorth York(Willowdale)South



M1PScarborough(Dorset Park / Wexford Heights / Scarborough Town Centre)

 
M2PNorth York(York Mills West)



M1RScarborough(Wexford / Maryvale)

 
M2RNorth York(Willowdale)West



M1SScarborough(Agincourt) 

 
M2SNot assigned



M1TScarborough(Clarks Corners

IndexError: list index out of range

In [ ]:
# grouping rows with same borough and neighbor
df=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

# 3. printing shape of dataframe

In [ ]:
df.shape

# part 2. get geographical coordinates of boroughs

# 4. Reading data of latitude and longitude into dataframe

In [ ]:
loc=pd.read_csv('https://cocl.us/Geospatial_data')
loc.head()

In [ ]:
loc.rename(columns={'Postal Code':'Postcode'},inplace=True)

# 5. merging the dataframes

In [ ]:
new_df=df.merge(loc,on='Postcode',how='right')

In [ ]:
new_df.tail()

# Part 3 Explore and clusters areas in toronto

# get coordinates of toronto

In [ ]:
address = 'Toronto'
geolocator = Nominatim(user_agent="my")
location = geolocator.geocode(address,timeout=15)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# create a map of various places of toronto from data frame

In [ ]:
Toronto_map=folium.Map(location=[latitude,longitude])
for bor,neigh,lat,long in zip(new_df['Borough'],new_df['Neighborhood'],new_df['Latitude'],new_df['Longitude']):
    label='{},{}'.format(neigh,bor)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3572as',
                       fill_opacity=0.8).add_to(Toronto_map)
Toronto_map

# get a list of unique boroughs that contain "toronto" in borough name

In [ ]:
unique_borough=set(new_df['Borough'])
l=[]
for b in unique_borough:
    if 'toronto' in b.lower():
        l.append(b)
l

# crate new data frame with boroughs that contain "toronto" in borough name

In [ ]:
new_df=new_df[new_df['Borough'].isin(l)].reset_index(drop=True)
new_df.head()
new_df.shape

# create a map of toronto and mark places as in new data frame

In [ ]:
map_toronto=folium.Map(location=[latitude,longitude])
for bor,neigh,lat,long in zip(new_df['Borough'],new_df['Neighborhood'],new_df['Latitude'],new_df['Longitude']):
    label='{}, {}'.format(neigh,bor)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3572as',
                       fill_opacity=0.8).add_to(map_toronto)
map_toronto

# foursquare api credentials

In [ ]:
CLIENT_ID = 'id' # Foursquare ID
CLIENT_SECRET = 'secret' # Foursquare Secret
VERSION = '20200314' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# function to extract the category of the venue

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# get venues returned by foursquare with in 500 meters

In [ ]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=new_df['Neighborhood'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )



In [ ]:
toronto_venues.head()

# to get how many venues are there for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

# Analyzing each area

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

In [ ]:
# function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# get top 10 venues and storing its details in a data frame

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# making clusters using k-means

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = new_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

# visualising clusters returned by k-means

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining clusters

## Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]